## 指定Token

In [10]:
TOKEN = '<SECRET KEY>'

## 使用yt-dlp 下載 Youtube 影片

In [2]:
!pip install yt-dlp


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [7]:
import yt_dlp

# 設定選項
ydl_opts = {
    'format': 'bestaudio/best',
    'outtmpl': 'joeman',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'mp3',
        'preferredquality': '192',
    }],
}

# 填入影片的URL
url = 'https://www.youtube.com/watch?v=PRp3kJtMQwU'

# 建立 yt_dlp 下載器物件
with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    ydl.download([url])


[youtube] Extracting URL: https://www.youtube.com/watch?v=PRp3kJtMQwU
[youtube] PRp3kJtMQwU: Downloading webpage
[youtube] PRp3kJtMQwU: Downloading android player API JSON
[info] PRp3kJtMQwU: Downloading 1 format(s): 251
[dashsegments] Total fragments: 3
[download] Destination: joeman
[download] 100% of   22.96MiB in 00:00:02 at 9.26MiB/s               
[ExtractAudio] Destination: joeman.mp3
Deleting original file joeman (pass -k to keep)


In [8]:
! open joeman.mp3

## 使用Whisper API 將音檔轉換成文字

In [18]:
! pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 678.6 kB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [21]:
import os
import openai
openai.api_key = TOKEN
audio_file = open("joeman.mp3", "rb")
transcript = openai.Audio.transcribe("whisper-1", audio_file)


APIError: Maximum content size limit (26214400) exceeded (33891014 bytes read) {
  "error": {
    "message": "Maximum content size limit (26214400) exceeded (33891014 bytes read)",
    "type": "server_error",
    "param": null,
    "code": null
  }
} 413 {'error': {'message': 'Maximum content size limit (26214400) exceeded (33891014 bytes read)', 'type': 'server_error', 'param': None, 'code': None}} {'Date': 'Sun, 05 Mar 2023 08:20:56 GMT', 'Content-Type': 'application/json', 'Content-Length': '171', 'Connection': 'keep-alive', 'Openai-Processing-Ms': '319', 'Openai-Version': '2020-10-01', 'Strict-Transport-Security': 'max-age=15724800; includeSubDomains', 'X-Request-Id': 'c10abcb8a9246d448d2cbee97163fba3'}

## 使用pydub  分割音檔

In [33]:
! pip install pydub


[notice] A new release of pip is available: 23.0 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


In [34]:
from pydub import AudioSegment

# 載入 MP3 音檔
sound = AudioSegment.from_file('joeman.mp3', format='mp3')

# 設定每個分割檔案的長度（單位：毫秒）
segment_length = 1000000

# 將音檔分割成多個檔案
for i, chunk in enumerate(sound[::segment_length]):
    # 設定分割檔案的檔名
    chunk.export(f'output_{i}.mp3', format='mp3')


In [35]:
import os
import openai
openai.api_key = TOKEN
audio_file = open("output_0.mp3", "rb")
transcript_0 = openai.Audio.transcribe("whisper-1", audio_file)


In [38]:
#transcript_0.to_dict().get('text')

In [39]:
audio_file = open("output_1.mp3", "rb")
transcript_1 = openai.Audio.transcribe("whisper-1", audio_file)

In [41]:
# transcript_1.to_dict().get('text')

In [42]:
transcript = transcript_0.to_dict().get('text') + ' ' + transcript_1.to_dict().get('text')

In [44]:
#transcript

## 使用 ChatGPT API  摘要文章

In [46]:
import os
import openai
openai.api_key = TOKEN

completion = openai.ChatCompletion.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "請你成為文章摘要的小幫手，摘要以下文字，以繁體中文輸出"},
    {"role": "user", "content": transcript}
  ]
)

print(completion.choices[0].message)


InvalidRequestError: This model's maximum context length is 4096 tokens. However, your messages resulted in 10916 tokens. Please reduce the length of the messages.

In [49]:
transcript_ary = []

ret = ''
for script in transcript.split():
    ret = ret + ' ' + script
    if len(ret) > 1000:
        transcript_ary.append(ret)
        ret = ''
transcript_ary.append(ret)

In [50]:
len(transcript_ary)

9

In [51]:
import os
import openai
openai.api_key = TOKEN

result_ary = []

for t in transcript_ary:
    completion = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "system", "content": "請你成為文章摘要的小幫手，摘要以下文字，以繁體中文輸出"},
        {"role": "user", "content": t}
      ]
    )

    result_ary.append(completion.choices[0].message)


In [56]:
len(result_ary)

9

In [58]:
result_ary[0].get('content')

'這篇文章是關於一位專長於日本房地產投資的老師，他分享了自己在金融風暴後決定進軍日本房地產的經歷和心得。他提到金融世界的風險很高，讓他開始思考換實際的具體投資，並選擇投資在日本的房地產市場。他也分享了自己虧損的經驗，並表示要成功投資需要精準的觀察力。另外，他還提到台灣的房地產市場也有很多好處，但他當時覺得日本的投資機會更大，所以決定前往投資。'

In [60]:
for res in result_ary:
    print()
    print(res.get('content'))


這篇文章是關於一位專長於日本房地產投資的老師，他分享了自己在金融風暴後決定進軍日本房地產的經歷和心得。他提到金融世界的風險很高，讓他開始思考換實際的具體投資，並選擇投資在日本的房地產市場。他也分享了自己虧損的經驗，並表示要成功投資需要精準的觀察力。另外，他還提到台灣的房地產市場也有很多好處，但他當時覺得日本的投資機會更大，所以決定前往投資。

這段文字講述作者在投資日本房地產時，因為公司設立、稅務負擔等問題而感到困難。另外，作者提到日本房地產投資不易，需要大量貸款，而且要被迫每年買物件。作者也分享自己投資的經驗，認為居酒屋是一種比較穩定的投資對象。最後，作者建議投資者要謹慎選擇物件，並找專業人士協助整理房源資訊。

本文介紹了在東京大學附近稻田附近的京區，它是一個很好的文教區，而且比較穩定的區域在西部的丘陵區吉祥寺附近，其中的Lions Mountains住宅區有7.5萬日元的租金收入，年回報率為5.8%，但投資者需要注意租戶的風險，尤其是租給學生的話，風險更高。該地區的房屋需求較穩定的上班族為佳，家庭更不容易搬走。

文章提到了購買日本房產的注意事項和優缺點。建議購買2LDK以上的房子，避免租客搬家帶來的麻煩，以及注意租客遺留物品的問題。營業場合租約可以長達10年以上，但修繕費用成本較高。文章也介紹了投資日本房產的漲幅和增值潛力，以及老房子的實際優點和稅賦成本。

本文介紹了作者在日本買房的經驗和投報分析。文章提到日本的房市雖然漲幅不及台灣大，但也有漲幅，且持有成本高，稅收也會影響房價漲幅。文章建議買房要選擇人口集中的區域，並要避免套房，貸租約的營業場合是穩定的選擇。最後，作者分享了自己曾買的一個房子，雖然在風化區但租金很高，且經過大地震和其他事件後價格仍穩定。

這段文字主要是講述在日本購買房產需要注意的事項，例如在購買前要實際去看一下房子的狀況，以避免買到風化區等不良地區。另外，要注意房屋是否有修繕基金，以及管理費是否存了一大筆錢作為嫁妝。修繕基金與管理費適量較好，過多會影響投報率，但若購買的是1996年出租的舊房，則可能會有貴到沒有改的管理費。最後，作者提到了一個例子，警示讀者要注意租金與管理費之間的投報率，以免造成損失。

本文主要提到房地產投資的注意事項，不要只听从房地產經紀人的建议，要作出自己的判断，尤其是在選擇地點和物業類型上，要注意考慮租客換手率